In [ ]:
# Standardization of mechanical property units
import pandas as pd
import re

# 📁 输入输出路径
input_csv  = r"Your_split_pdfs_Path_Extraction_Result_1.csv"
output_csv = r"Your_split_pdfs_Path_Extraction_Result_Standardize_1.csv"

# 📌 字段定义
mechanical_mpa_fields = ["Tensile Strength", "Young's Modulus", "Flexural Modulus"]
percent_fields = ["Elongation at Break"]
temperature_k_fields = ["Glass Transition", "Melting Point"]

# 🚀 力学性能字段单位转换（统一为 MPa）
def convert_mech(val):
    if pd.isna(val):
        return pd.NA
    s = str(val).strip()

    # ✅ 标准化常见单位表达
    s = s.replace("N/mm²", "MPa").replace("n/mm²", "mpa")
    s = s.replace("N/mm2", "MPa").replace("n/mm2", "mpa")
    s = s.replace("MN/m²", "MPa").replace("mn/m²", "mpa")
    s = s.replace("N/m²", "Pa").replace("N/m2", "Pa")
    s = s.replace("n/m²", "Pa").replace("n/m2", "Pa")

    # ✅ 提取数值
    m = re.search(r'-?\d+(?:\.\d+)?', s)
    if not m:
        return pd.NA
    num = float(m.group())
    unit = s.lower()

    # ✅ 换算逻辑
    if 'gpa' in unit:
        return num * 1000
    elif 'mpa' in unit:
        return num
    elif 'kpa' in unit:
        return num / 1000
    elif 'pa' in unit:
        return num / 1e6
    elif 'cn/dtex' in unit:
        return num * 0.088055  # 1 cN/dtex ≈ 0.088 MPa
    elif 'cn/tex' in unit:
        return num * 0.1
    elif 'n/tex' in unit:
        return num * 100
    elif 'dyn/cm2' in unit or 'dyn/cm²' in unit:
        return num / 1e7
    else:
        return num  # 无单位默认 MPa

# ⚡ 冲击强度单位转换（统一为 kJ/m²）
def convert_impact(val):
    if pd.isna(val):
        return pd.NA
    s = str(val).lower().strip()

    # 提取数值
    m = re.search(r'-?\d+(?:\.\d+)?', s)
    if not m:
        return pd.NA
    num = float(m.group())

    # 单位换算
    if 'j/cm²' in s or 'j/cm2' in s:
        return num * 10  # 1 J/cm² = 10 kJ/m²
    elif 'kj/m²' in s or 'kj/m2' in s:
        return num
    elif 'j/m²' in s or 'j/m2' in s:
        return num / 1000
    else:
        return num  # 默认按 kJ/m² 处理


# 🌡️ 温度字段转换（K → °C），剥除 °C/℃/K 等单位符号
def convert_temp(val):
    if pd.isna(val):
        return pd.NA
    s = str(val).strip()

    # ✅ 删除所有 °、℃、K、C、F，允许中间有空格
    s = re.sub(r'\s*(°\s*[CFK]|℃|[CFK])\s*', '', s, flags=re.IGNORECASE)

    # 提取数值
    m = re.search(r'-?\d+(?:\.\d+)?', s)
    if not m:
        return pd.NA
    num = float(m.group())

    # ✅ 判断原始字符串是否含“ K ”（不能是“°K”）
    if re.search(r'(?<!°)\bK\b', str(val), re.IGNORECASE):
        return num - 273.15
    else:
        return num



# 百分比字段提取数字
def extract_numeric(val):
    s = str(val)
    m = re.match(r'(-?\d+(?:\.\d+)?)', s.strip())
    return float(m.group(1)) if m else pd.NA

# ❌ 清除年份 / 月份单元格
def should_remove(val: str):
    val = str(val).strip()
    if re.fullmatch(r"(19|20)\d{2}", val):
        return True
    if re.search(r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\b', val, re.IGNORECASE):
        return True
    if re.search(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\b', val, re.IGNORECASE):
        return True
    return False

# 🧹 执行清洗
df = pd.read_csv(input_csv, dtype=str)
removed_cells = []

# 年份和月份剔除
for col in df.columns:
    for i, val in df[col].items():
        if pd.notna(val) and should_remove(val):
            removed_cells.append((i+2, col, val))
            df.at[i, col] = pd.NA

# ✅ 自动识别 Impact Strength 字段名
impact_fields = [col for col in df.columns if re.search(r'impact\s*strength', col, re.IGNORECASE)]
for col in impact_fields:
    df[col] = df[col].apply(convert_impact)


# 应用单位转换
for col in mechanical_mpa_fields:
    if col in df.columns:
        df[col] = df[col].apply(convert_mech)

for col in percent_fields:
    if col in df.columns:
        df[col] = df[col].apply(extract_numeric)

for col in temperature_k_fields:
    if col in df.columns:
        df[col] = df[col].apply(convert_temp)

rename_map = {
    "Tensile Strength": "Tensile Strength (MPa)",
    "Young's Modulus": "Young's Modulus (MPa)",
    "Flexural Modulus": "Flexural Modulus (MPa)",
    "Elongation at Break": "Elongation at Break (%)",
    "Glass Transition": "Glass Transition Temperature (°C)",
    "Melting Point": "Melting Temperature (°C)",
}
df.rename(columns=rename_map, inplace=True)


# 💾 保存清洗后结果
df.to_csv(output_csv, index=False, encoding="utf-8-sig")

# 📊 日志输出
print(f"✅ 清洗完成，保存至：{output_csv}")
print(f"🧹 共清空年份/月类单元格：{len(removed_cells)}")
if removed_cells:
    print("📌 示例（前10条）：")
    for i, (row, col, val) in enumerate(removed_cells[:10], 1):
        print(f"{i}. 行 {row}，列 {col}，原内容：'{val}'")


In [ ]:
# Standardization of swelling-ratio units
import pandas as pd
import re

# 📁 输入输出路径
input_csv  = r"Your_split_pdfs_Path_Extraction_Result_2.csv"
output_csv = r"Your_split_pdfs_Path_Extraction_Result_Standardize_2.csv"

# 📌 字段定义
percent_fields = ["Swelling Ratio"]
temperature_k_fields = ["Glass Transition", "Melting Point"]

# 🌡️ 温度字段转换（K → °C），剥除 °C/℃/K 等单位符号
def convert_temp(val):
    if pd.isna(val):
        return pd.NA
    s = str(val).strip()
    s = re.sub(r'\s*(°\s*[CFK]|℃|[CFK])\s*', '', s, flags=re.IGNORECASE)
    m = re.search(r'-?\d+(?:\.\d+)?', s)
    if not m:
        return pd.NA
    num = float(m.group())
    if re.search(r'(?<!°)\bK\b', str(val), re.IGNORECASE):
        return num - 273.15
    else:
        return num

# ➗ 百分比处理 + g/g删除 + 区间取前值
def extract_swelling(val):
    if pd.isna(val):
        return pd.NA
    s = str(val).strip()

    # 删除包含 g/g 的
    if re.search(r'g\s*/\s*g', s, re.IGNORECASE):
        return pd.NA

    # 匹配范围型值如 100–200 / 100-200，保留第一个数
    m = re.match(r'(-?\d+(?:\.\d+)?)(?:\s*[-–~～]\s*\d+(?:\.\d+)?)*', s)
    if m:
        num = float(m.group(1))
    else:
        return pd.NA

    # 如果原始字符串包含 %，除以100
    if '%' in s:
        return num / 100
    else:
        return num

# ❌ 清除年份 / 月份单元格
def should_remove(val: str):
    val = str(val).strip()
    if re.fullmatch(r"(19|20)\d{2}", val):
        return True
    if re.search(r'\b(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\b', val, re.IGNORECASE):
        return True
    if re.search(r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\b', val, re.IGNORECASE):
        return True
    return False

# 🧹 执行清洗
df = pd.read_csv(input_csv, dtype=str)
removed_cells = []

# 年份和月份剔除
for col in df.columns:
    for i, val in df[col].items():
        if pd.notna(val) and should_remove(val):
            removed_cells.append((i+2, col, val))
            df.at[i, col] = pd.NA

# 应用单位转换
for col in percent_fields:
    if col in df.columns:
        df[col] = df[col].apply(extract_swelling)

for col in temperature_k_fields:
    if col in df.columns:
        df[col] = df[col].apply(convert_temp)

# ✅ 字段重命名
rename_map = {
    "Swelling Ratio": "Swelling Ratio (times)",
    "Glass Transition": "Glass Transition Temperature (°C)",
    "Melting Point": "Melting Temperature (°C)",
}
df.rename(columns=rename_map, inplace=True)

# 📈 统计 Swelling Ratio 含有效数字的单元格数量
swelling_col = "Swelling Ratio (times)"
if swelling_col in df.columns:
    count_numeric = df[swelling_col].apply(lambda x: pd.notna(x) and isinstance(x, (int, float, complex))).sum()
else:
    count_numeric = 0

# 💾 保存清洗后结果
df.to_csv(output_csv, index=False, encoding="utf-8-sig")

# 📊 日志输出
print(f"✅ 清洗完成，保存至：{output_csv}")
print(f"🧹 共清空年份/月类单元格：{len(removed_cells)}")
if removed_cells:
    print("📌 示例（前10条）：")
    for i, (row, col, val) in enumerate(removed_cells[:10], 1):
        print(f"{i}. 行 {row}，列 {col}，原内容：'{val}'")

print(f"📊 Swelling Ratio (times) 含有效数值的单元格数：{count_numeric}")
